# Brain Imaging Data Structure (BIDS)

---

## When NIfTI is not enough

We've only worked with a single T1w scan from a single subject. However, neuraoimaging research requires working with *different scan modalities* across *several subjects*.

Let's take a look at the rest of our NIfTI dataset from the last lesson. Imagine you are a collaborator who's currently gotten your hands on this dataset for an analysis. What are some immediate questions you might have that aren't apparent at a first glance?

In [10]:
import seedir as sd

sd.seedir('data/dicom_examples/nii', style='emoji')

📁 nii/
├─📄 dcm_anat_ses-01_scout_20190219111436_1.json
├─📄 dcm_anat_ses-01_scout_20190219111436_1.nii.gz
├─📄 dcm_anat_ses-01_scout_20190219111436_2_i00001.json
├─📄 dcm_anat_ses-01_scout_20190219111436_2_i00001.nii.gz
├─📄 dcm_anat_ses-01_scout_20190219111436_2_i00002.json
├─📄 dcm_anat_ses-01_scout_20190219111436_2_i00002.nii.gz
├─📄 dcm_anat_ses-01_scout_20190219111436_2_i00003.json
├─📄 dcm_anat_ses-01_scout_20190219111436_2_i00003.nii.gz
├─📄 dcm_anat_ses-01_scout_20190219111436_2_i00004.json
├─📄 dcm_anat_ses-01_scout_20190219111436_2_i00004.nii.gz
├─📄 dcm_anat_ses-01_scout_20190219111436_3_i00001.json
├─📄 dcm_anat_ses-01_scout_20190219111436_3_i00001.nii.gz
├─📄 dcm_anat_ses-01_scout_20190219111436_3_i00002.json
├─📄 dcm_anat_ses-01_scout_20190219111436_3_i00002.nii.gz
├─📄 dcm_anat_ses-01_scout_20190219111436_3_i00003.json
├─📄 dcm_anat_ses-01_scout_20190219111436_3_i00003.nii.gz
├─📄 dcm_anat_ses-01_scout_20190219111436_4_i00001.json
├─📄 dcm_anat_ses-01_scout_20190219111436_4_i00001.nii.gz

* How many subjects were scanned?
* What are the demographics of the participants?
* What is the name of the dataset?
* Who are the authors of the dataset?
* Is there a citation I should use to give credit to the authors?
* Is there a more elaborate description of the fMRI tasks?
* Do any of the fMRI tasks have an event-related design?
* What do the fMRI task stimuli look like?

## Introducing BIDS

The [Brain Imaging Data Structure (BIDS)](https://www.nature.com/articles/sdata201644) is a simple and intuitive way to organize and describe your neuroimaging and behavioural data. Neuroimaging experiments result in complicated data that can be arranged in several different ways. BIDS tackles this problem by suggesting a new standard for the arrangement of neuroimaging datasets. Using the same organizational standard for all of your studies will also allow you to easily reuse your scripts and share data and code with other researchers.

For this lesson, we'll be focusing on organizing MRI data.

For a more comprehensive overview, check out the [BIDS Starter Kit](https://github.com/bids-standard/bids-starter-kit/wiki).

![BIDS example](imgs/bids_structure.webp)

### Structure

The current BIDS specification is [documented online](https://bids-specification.readthedocs.io/en/stable/).

```
dataset/
    rawdata/
        dataset_description.json
        participants.tsv
        CHANGES
        READEME
        task-rest.json
        sub-01/
            ses-01/
                anat/
                func/
        sub-02/
        ...
    sourcedata/
        ...
    derivatives/
        pipeline_1/
        pipeline_2/
        ...
    code/
    stimuli/
```

Let us converting the NIfTI data to BIDS manually.

### Conversion Methods

* [Dcm2Bids](https://github.com/cbedetti/Dcm2Bids)

You can use `dcm2bids_scaffold` to help you build the bids structure automatically. `dcm2bids_scaffold` creates basic BIDS files and directories in the output directory (by default folder where the script is launched).

In [11]:
import os
import importlib.util

package_name = ['dcm2bids']

for i in package_name:
    spec = importlib.util.find_spec(i)
    if spec is None:
        print(i + ' is not installed.')
        os.system('pip install ' + i)
        print(i + 'installed.')
    else:
        print(i + ' is installed.')

os.system('dcm2bids_scaffold -o data/dicom_examples/bids/')

dcm2bids is installed.


0

In [12]:
import json
from collections import OrderedDict

data = OrderedDict()
data['Name'] = 'BIDS Test'
data['BIDSVersion'] = '1.6.0'
data['License'] = ''
data['Authors'] = ['', '', '']
data['Acknowledgements'] = ''
data['HowToAcknowledge'] = ''
data['Funding'] = ['']
data['ReferenceAndLinks'] = ['']
dataset_json_name = 'data/dicom_examples/bids/dataset_description.json'

with open(dataset_json_name, 'w') as f:
    json.dump(data, f, sort_keys=False, indent=4)

To build the configuration file, you need to have an example of the sidecars. You can use `dcm2bids_helper` with the DICOMs of one participant. It will launch `dcm2niix` and save the result inside the `tmp_dcm2bids/helper` of the output directory.

In [ ]:
# There is an error of output path on Windows system
# Try to do that on Linux
os.system('dcm2bids_helper -d data/dicom_examples/0219191_mystudy-0219-1114/dcm -o output/')

We will add the configuration file to `bids/code` to help the generation of bids structure. The detailed explanation can be seen on [Configuration file](https://unfmontreal.github.io/Dcm2Bids/docs/3-configuration/). Here we can just add the json below to the directory.

> bids/code/bids_config.json

```
{
  "descriptions": [
    {
      "dataType": "anat",
      "modalityLabel": "T1w",
      "criteria": {
        "SeriesDescription": "anat_ses-01_T1w"
      }
    },
    {
      "dataType": "func",
      "modalityLabel": "bold",
      "customLabels": "task-sound",
      "criteria": {
        "SeriesDescription": "func_ses-01_task-sound"
      }
    },
    {
      "dataType": "func",
      "modalityLabel": "bold",
      "customLabels": "task-story",
      "criteria": {
        "SeriesDescription": "func_ses-01_task-story"
      }
    },
    {
      "dataType": "func",
      "modalityLabel": "bold",
      "customLabels": "task-faces",
      "criteria": {
        "SeriesDescription": "func_ses-01_task-faces*"
      }
    }
  ]
}
```

And then we can use `dcm2bids` to construct our final bids.

In [14]:
# There is an error of output path on Windows system
# Try to do that on Linux

os.system(" dcm2bids -d data/dicom_examples/0219191_mystudy-0219-1114/dcm/ -p 01 -s 01 -c data/dicom_examples/bids/code/bids_config.json -o data/dicom_examples/bids/")

1

The `sub-01` directory and `tmp_dcm2bids` are raw data files. You can compare the size of the previous `data/
nii` folder. Two of which are the same.

## BIDS Validator

Can be run [online](https://bids-standard.github.io/bids-validator).

Select the data directory and wait for it to finish validation.

View errors and warnings.